In [1]:
!pip install -r requirements.txt

You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

In [3]:
'''
HateSpeechDataset Features:
count: number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF)
hate_speech: number of CF users who judged the tweet to be hate speech
offensive_language: number of CF users who judged the tweet to be offensive
neither: number of CF users who judged the tweet to be neither offensive nor non-offensive
class: class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither
tweet: text tweet
'''
HateSpeechDataset = pd.read_csv('datasets/Hate_Speech_and_Offensive_Language_Dataset.csv')

'''
DynamicallyGeneratedDataset Features:
The features were not explained in Kaggle
'''
DynamicallyGeneratedDataset = pd.read_csv('datasets\Dynamically_Generated_Hate_Dataset_Entries.csv')


In [4]:
display(HateSpeechDataset.sample(10))
display(HateSpeechDataset.info())
display(HateSpeechDataset.describe())

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
18936,19355,3,1,2,0,1,"RT @duttyOGshake: ""pussy ass nigga"" http://t.c..."
15044,15401,3,0,3,0,1,RT @DragonflyJonez: She from the D. She might ...
10817,11098,3,0,1,2,2,I pictured Angus running up the side of FOB Sh...
2704,2768,3,2,1,0,0,@Buckm00se Joe Cortez is the ultimate faggot t...
12125,12434,3,0,1,2,2,Just got done taking some photos with brownie ...
8667,8906,3,0,3,0,1,"Dawg, what the fuck happened to ""so and so"" wh..."
18478,18887,3,0,3,0,1,"RT @_naptural_: ""@NoBeeetch: When you drop all..."
8758,9002,3,0,0,3,2,Do girls wear bathrobes still cuz I have reall...
23117,23601,3,0,0,3,2,Yea buddy RT @YallJustDontKno: A lot of NBA tr...
22674,23152,3,0,3,0,1,When a chicks head is weak I just close my eye...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


None

,Unnamed: 0,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,12681.192027,3.243473,0.280515,2.413711,0.549247,1.110277
std,7299.553863,0.883060,0.631851,1.399459,1.113299,0.462089
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,6372.500000,3.000000,0.000000,2.000000,0.000000,1.000000
50%,12703.000000,3.000000,0.000000,3.000000,0.000000,1.000000
75%,18995.500000,3.000000,0.000000,3.000000,0.000000,1.000000
max,25296.000000,9.000000,7.000000,9.000000,9.000000,2.000000


In [5]:
display(DynamicallyGeneratedDataset.sample(10))
display(DynamicallyGeneratedDataset.info())
display(DynamicallyGeneratedDataset.describe())

,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
7983,8034,20682p,it makes me so proud to be british born and no...,nothate,none,NaN,NaN,perturbation,2b,train,tbubpfn6ip
28502,28569,32631p,"Fuck you & fuck your family""š this was the mo...",hate,derogation,NaN,NaN,perturbation,3b,train,qiokkci7f8
30173,30241,33548p,Why do some women act as if they were not wort...,nothate,none,NaN,NaN,perturbation,3b,train,trrf46jwfp
23494,23559,29114p,No one loves fucking green tea,nothate,none,NaN,NaN,perturbation,2b,train,vde7gn0nrl
23943,24008,29344p,lets talk about the antithesis of selfishness...,nothate,none,NaN,NaN,perturbation,2b,train,tbubpfn6ip
36960,37043,5886,The disabled kids in mainstream school ruin it...,hate,notgiven,True,0.98530,dynabench entry,1,train,gnzuctwed3
35559,35635,46896,hey people she has been given a platform by th...,nothate,none,False,0.99992,dynabench entry,3a,train,vde7gn0nrl
39060,39163,8351,bananas are 4 black people,hate,notgiven,True,0.98408,dynabench entry,1,train,elgzdd8tvb
8456,8507,20929p,"he has a black heart, all blacks are black in ...",hate,derogation,NaN,NaN,perturbation,2b,train,elgzdd8tvb
9867,9918,21649,don't fucking call me a chink,nothate,none,False,0.98513,dynabench entry,2a,train,tbubpfn6ip


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40623 entries, 0 to 40622
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      40623 non-null  int64  
 1   id              40623 non-null  object 
 2   text            40623 non-null  object 
 3   label           40623 non-null  object 
 4   type            40623 non-null  object 
 5   model_wrong     26097 non-null  object 
 6   db.model_preds  26097 non-null  float64
 7   status          40623 non-null  object 
 8   round           40623 non-null  object 
 9   split           40623 non-null  object 
 10  annotator       40623 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 3.4+ MB


None

,Unnamed: 0,db.model_preds
count,40623.000000,26097.000000
mean,20371.983236,0.585455
std,11743.590183,0.448801
min,1.000000,0.000010
25%,10206.500000,0.036560
50%,20374.000000,0.910290
75%,30534.500000,0.995500
max,40740.000000,0.999920


In [6]:
DynamicallyGeneratedDataset['text']

0        It should be legal for a husband to rape his wife
1                                  dalits are lowlife scum
2                                      dalits are lowlives
3        It was a better world when women dare not ques...
4        I don't work this hard so that those immigrant...
                               ...                        
40618    we should forcibly remove all the foreigners f...
40619    we should forcibly remove all the immigrants f...
40620                 dalits are unworthy of the Gods love
40621                                 dalits are worthless
40622         dalits are deserves to be treated as lowlife
Name: text, Length: 40623, dtype: object